In [1]:
import numpy as np
import pandas as pd

### Parameter Grid 기초

In [2]:
from sklearn.model_selection import ParameterGrid

grid = {"n_neighbors": [3, 5, 7],
        "metric": ["Manhattan", "Euclidean"]}

#ParameterGrid(grid) # 이부분만 돌리면 알아보기 힘듬(generator 형태), 하지만 list를 감싸서 수행하는 것은 아님
list(ParameterGrid(grid))

[{'metric': 'Manhattan', 'n_neighbors': 3},
 {'metric': 'Manhattan', 'n_neighbors': 5},
 {'metric': 'Manhattan', 'n_neighbors': 7},
 {'metric': 'Euclidean', 'n_neighbors': 3},
 {'metric': 'Euclidean', 'n_neighbors': 5},
 {'metric': 'Euclidean', 'n_neighbors': 7}]

### 함수 입력값으로 dict 받는 방법 
func(<u>**</u>{'a':1, 'b':2})

In [3]:
def add_f(a, b):
    return a + b

input_f = {"a": 1, "b": 2} # a, b는 문자열인 것 인지
#input_f = {"b": 2, "a": 1} # 순서가 바뀌어도 상관 없음
add_f(**input_f)

3

### 그리드 서치 실습 예제
- 사용 데이터: iris dataset (sklearn 제공 데이터)<br><br>
- 사용 모델: (1) k-최근접 이웃
    - n_neighbors (3, 5, 7)
    - metric (euclidean, manhattan)<br><br>
- 사용 모델: (2) 서포트 벡터 머신
    - kernel: rbf, linear
    - C: 0.1, 1, 10
<br><br>
- 평가 척도: F1 score

In [4]:
# 예제 데이터 불러오기
from sklearn.datasets import load_iris
X = load_iris()['data'] # feature
Y = load_iris()['target'] # label

# 학습 데이터와 평가 데이터 분할
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y, random_state = 4321)

In [5]:
print('X : ')
print(pd.DataFrame(X).head())

print('Y : ')
print(pd.DataFrame(Y).head())

X : 
     0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
Y : 
   0
0  0
1  0
2  0
3  0
4  0


In [6]:
print('Train_X : {}'.format(Train_X.shape))
print('Test_X : {}'.format(Test_X.shape))
print('Train_Y : {}'.format(Train_Y.shape))
print('Test_Y : {}'.format(Test_Y.shape))

Train_X : (112, 4)
Test_X : (38, 4)
Train_Y : (112,)
Test_Y : (38,)


### 모델링 ( KNN, SVM )

In [7]:
# 모델 불러오기
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC

In [8]:
# 파라미터 그리드 생성
param_grid = dict() 
# 입력: 모델 함수, 출력: 모델의 하이퍼 파라미터 그리드

# 모델별 파라미터 그리드 생성
param_grid_for_knn = ParameterGrid({"n_neighbors": [3, 5, 7],
                           "metric":['euclidean', 'manhattan']})

param_grid_for_svm = ParameterGrid({"C": [0.1, 1, 10],
                           "kernel":['rbf', 'linear']})

# 모델 - 하이퍼 파라미터 그리드를 param_grid에 추가
param_grid[KNN] = param_grid_for_knn
param_grid[SVC] = param_grid_for_svm

In [9]:
# 하이퍼 파라미터 튜닝 
best_score = -1 # 현재까지 찾은 가장 높은 f1_score (f1 score는 절대 0보다 작을수 없기에, -1로 설정해도 무방)

from tqdm import tqdm # for문의 진행상태를 알려주는 루틴
from sklearn.metrics import f1_score

for model_func, model_name in [(KNN, 'KNN'), (SVC, 'SVC')]: # model 먼저 
    
    for param in tqdm(param_grid[model_func]): # grid 뒤에 
        
        model = model_func(**param).fit(Train_X, Train_Y)
        pred_Y = model.predict(Test_X)        
        score = f1_score(Test_Y, pred_Y, average = 'micro') # multiclass
        
        print('model: {}, parameter : {}'.format(model_name, param))
        print('현재 스코어 : ',score, 'vs 최고 스코어 : ', best_score)
        print('-------'*10)
        
        pd.DataFrame({'x' : })
        
        if score > best_score: 
            # 현재 점수가 지금까지 찾은 최고 점수보다 좋으면, 최고 모델, 파라미터, 점수 업데이트
            best_model_name = model_name
            best_model_func = model_func
            best_score = score
            best_param = param
            
            # best_model = model

100%|██████████| 6/6 [00:00<00:00, 318.79it/s]

model: KNN, parameter : {'metric': 'euclidean', 'n_neighbors': 3}
현재 스코어 :  0.9473684210526315 vs 최고 스코어 :  -1
----------------------------------------------------------------------
model: KNN, parameter : {'metric': 'euclidean', 'n_neighbors': 5}
현재 스코어 :  0.9210526315789473 vs 최고 스코어 :  0.9473684210526315
----------------------------------------------------------------------
model: KNN, parameter : {'metric': 'euclidean', 'n_neighbors': 7}
현재 스코어 :  0.9736842105263158 vs 최고 스코어 :  0.9473684210526315
----------------------------------------------------------------------
model: KNN, parameter : {'metric': 'manhattan', 'n_neighbors': 3}
현재 스코어 :  0.9210526315789473 vs 최고 스코어 :  0.9736842105263158
----------------------------------------------------------------------
model: KNN, parameter : {'metric': 'manhattan', 'n_neighbors': 5}
현재 스코어 :  0.9210526315789473 vs 최고 스코어 :  0.9736842105263158
----------------------------------------------------------------------
model: KNN, parameter : {'

In [10]:
print("모델:", best_model_name)
print("점수:", best_score)
print("파라미터:", best_param)

모델: KNN
점수: 0.9736842105263158
파라미터: {'metric': 'euclidean', 'n_neighbors': 7}


In [11]:
# 최종 모델 학습: 전체 X와 전체 Y에 대해.
final_model = best_model_func(**best_param).fit(X, Y)

In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Test_Y, pred_Y)

array([[13,  0,  0],
       [ 0, 10,  1],
       [ 0,  1, 13]])

In [13]:
# Scree-plot
## metric 결과값을 토대로 그래프를 그려 확인

import matplotlib
import matplotlib.pyplot as plt

print("Matplotlib version", matplotlib.__version__)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Matplotlib version 3.2.2


In [14]:
import seaborn as sns
print("Seaborn version : ", sns.__version__)
sns.set()
sns.set_style('whitegrid')
sns.set_color_codes()

Seaborn version :  0.10.1


In [ ]:
sns.replot